In [4]:
import sys
sys.path.append('C:/django/TheSilentVoice-signlanguagerecognition/signlanguage')
from neural_network import Network
from neural_network.layers import *
from neural_network.activations import *
from neural_network.optimizers import *
from neural_network.costs import *
from neural_network.metrics import *
from neural_network.utils import format_data

In [8]:
import os
import cv2
import numpy as np

def load_data(path, image_size):
    data = []
    labels = []
    categories = {
        "A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, "H": 7, "I": 8, "J": 9,
        "K": 10, "L": 11, "M": 12, "N": 13, "O": 14, "P": 15, "Q": 16, "R": 17, "S": 18,
        "T": 19, "U": 20, "V": 21, "W": 22, "X": 23, "Y": 24, "Z": 25, "del": 26,
        "nothing": 27, "space": 28
    }

    for category, label in categories.items():
        class_path = os.path.join(path, category)
        
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (image_size, image_size))
            image = image.astype('float32') / 255.0
            
            data.append(image)
            labels.append(label)
    
    return np.array(data), np.array(labels)


# Define the image size and paths
image_size = 100  # Adjust this as per your requirements
train_path = 'C:/Users/Dell/Downloads/GSASLdatasets/train/'
val_path = 'C:/Users/Dell/Downloads/GSASLdatasets/val/'
test_path = 'C:/Users/Dell/Downloads/GSASLdatasets/test/'

# Load training, validation, and test data
train_data, train_labels = load_data(train_path, image_size)
val_data, val_labels = load_data(val_path, image_size)
test_data, test_labels = load_data(test_path, image_size)

# Print the shapes of the loaded data
print("Train Data shape:", train_data.shape)
print("Train Labels shape:", train_labels.shape)
print("Validation Data shape:", val_data.shape)
print("Validation Labels shape:", val_labels.shape)
print("Test Data shape:", test_data.shape)
print("Test Labels shape:", test_labels.shape)


Train Data shape: (69600, 100, 100)
Train Labels shape: (69600,)
Validation Data shape: (8700, 100, 100)
Validation Labels shape: (8700,)
Test Data shape: (8700, 100, 100)
Test Labels shape: (8700,)


In [52]:
# Reshape the data for compatibility with neural network libraries
train_data = train_data.reshape((-1, image_size, image_size, 1))  # For a grayscale image
val_data = val_data.reshape((-1, image_size, image_size, 1))  # For a grayscale image
test_data = test_data.reshape((-1, image_size, image_size, 1))  # For a grayscale image


In [57]:
def one_hot_encode(labels, num_classes):
    encoded_labels = np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        encoded_labels[i][label] = 1
    return encoded_labels

num_classes = 29  # Total number of classes

train_labels_one_hot = one_hot_encode(train_labels, num_classes)
val_labels_one_hot = one_hot_encode(val_labels, num_classes)
test_labels_one_hot = one_hot_encode(test_labels, num_classes)

In [58]:
network = Network(
  optimizer=GradientDescentOptmizer(learning_rate=0.01),
  cost=BinaryCrossEntropyCost(),
  metric=CategoricalAccuracyMetric()
)

In [59]:

network.add(InputLayer(shape=(1, 100, 100)))
network.add(Conv2DLayer(num_of_kernels=64, kernel_shape=(3, 3), num_of_channels=1, stride=(1, 1)))
network.add(ActivationLayer(function=relu))
network.add(Conv2DLayer(num_of_kernels=64, kernel_shape=(3, 3), num_of_channels=1, stride=(1, 1)))
network.add(ActivationLayer(function=relu))
network.add(MaxPooling2DLayer(pool_shape=(2, 2), stride=(2, 2)))
network.add(DropoutLayer(drop_probability=0.2))

network.add(Conv2DLayer(num_of_kernels=128, kernel_shape=(3, 3), num_of_channels=1, stride=(1, 1)))
network.add(ActivationLayer(function=relu))
network.add(Conv2DLayer(num_of_kernels=128, kernel_shape=(3, 3), num_of_channels=1, stride=(1, 1)))
network.add(ActivationLayer(function=relu))
network.add(MaxPooling2DLayer(pool_shape=(2, 2), stride=(2, 2)))
network.add(DropoutLayer(drop_probability=0.3))

network.add(Conv2DLayer(num_of_kernels=256, kernel_shape=(3, 3), num_of_channels=1, stride=(1, 1)))
network.add(ActivationLayer(function=relu))
network.add(Conv2DLayer(num_of_kernels=256, kernel_shape=(3, 3), num_of_channels=1, stride=(1, 1)))
network.add(ActivationLayer(function=relu))
network.add(MaxPooling2DLayer(pool_shape=(2, 2), stride=(2, 2)))

network.add(DropoutLayer(drop_probability=0.4))

network.add(FlattenLayer())
network.add(DenseLayer(units=512))
network.add(ActivationLayer(function=relu))
network.add(DropoutLayer(drop_probability=0.3))
network.add(DenseLayer(units=256))
network.add(ActivationLayer(function=relu))
network.add(DropoutLayer(drop_probability=0.5))
network.add(DenseLayer(units=num_classes))
network.add(ActivationLayer(function=softmax))

# Load the network configuration
network.load_config('config.pkl.gz')

# Compile the network
network.compile()

In [61]:
network.fit(
    train_data=(train_data, train_labels_one_hot),
    epochs=15,
    batch_size=32,
    validation_data=(val_data, val_labels_one_hot),
    test_data=(test_data, test_labels_one_hot)
)


Training

Epoch: 1/15	2023-12-27 14:29:06.083772


MemoryError: Unable to allocate 2.59 GiB for an array with shape (69600, 100, 100, 1) and data type float32